In [1]:
import pandas as pd
import os
os.chdir('../utils')

from common_utils import *
from rsa import *

#### Rabin-Miller Test

In [6]:
primes = pd.read_csv('primes.csv', header=None, names=['prime'])
prime_list = primes['prime'].tolist()
# get 1000 random primes
sample_primes = primes.sample(10000)

In [7]:
# iterate over the sample primes
for index, row in sample_primes.iterrows():
    prime_test = int(row['prime'])
    if not miller_rabin(prime_test, 15):
        print('Failed on prime: ', prime_test)
        break
print('All primes passed the test')

All primes passed the test


In [9]:
composites = []

# generate 1000 composites
i = 18
while len(composites) < 1000:
    if i in prime_list:
        i += 1
        continue
    composites.append(i)
    i += 1

In [10]:
for i in composites:
    if miller_rabin(i, 15)[0]:
        print('Failed on composite: ', i)
        break
print('All composites passed the test')

All composites passed the test


#### Prime Generator Test

In [12]:
p = gen_prime(1024, 15) # 1024 is bits
print(p)

223851631610285018348631623282153293951091480257184522407417167991195422045811935202639683488923215254060857549414833049496911974528844683322824582205772133557387796650968198635360967604344225694960679721903396661165852725257374649542632356920940048255128981156849115443039058705159141409430418619394572205191


In [19]:
p = gen_prime_range(24, 32, 15) # 24 and 32 are not bits, but the range of numbers
print(p)

29


#### Vulnerable Key Generator Test

In [11]:
e, n, d, phi, p, q = gen_vulnerable_keys(nbits=1024, s=15)

In [12]:
print(n)
print(e)
print(d)    

731461097178306812144266922184339668058148325695542347184826567770946315844347912841299867808981800226096481096436993281030979746694253796076268432140266422812259932111974425360731045280990762036651318436381823678068395976909639079928401285761102455181284219236104827486380057882848642700540733098913695223957
158028985100711953797090038671899563412958968272587144741566524715581640717690124901903250963359420136923830603405290395923782425611172661973383976899034372416453625327031070155949777205749547283230858132813932699307175962373348215095219221012017783292816949285927972454260896288531028189103962767184415368451
55386319608426749298019273018709822078461655891082457657682095995876951601951


In [13]:
# check ed = 1 mod phi
print((e*d) % phi)

1


#### RSA Test

In [14]:
e, n, d, _, _, _ = gen_vulnerable_keys(nbits=1024, s=15)
message = "Hello, world!"
encrypted_message = encrypt(message, n, e)
print(encrypted_message)

403253439198194600926499197706056306142304595927576332635612605589193400850240584386200591943012199515257007524523587683758439422361683455979932299006062878550084586407163353431941916896479278829587945778553957478062115907442513235329201966546380033353414027347127051658936491632674215325743943344660225441659 745705436667553985693536706492981357626183156999283149442670002181381993409049134342927368194768854328309707475358653532972023870099934553352419296729900622059031840091526266692183512926269609428596329741740631956603847612962657154561893371562924521428902124975704838818284958608544500819205240963767340143428 658149200551296852625065288308549859183205240017913840404097021605385665511597303571932428549626826268932247617167789073873547199275029964898580312470568589262505860147593120064709287451498192988874126141188950432585794973282246927959306260918749368202089006187100171536671765471784853603464739391770037595431 7876804901050142129423224471247632241100609638222208861186779553813581

In [15]:
decrypted_message = decrypt(encrypted_message, n, d)
print(decrypted_message)

Hello, world !


#### Wiener Attack Test

In [8]:
e, n, d, _, _, _ = gen_vulnerable_keys(nbits=1024, s=15)
message = "Cryptography is the practice and study of techniques for secure communication in the presence of third parties called adversaries."
encrypted_message = encrypt(message, n, e)
print(encrypted_message)

...


In [9]:
potential_private_keys = wiener_atk(e, n, d) # d is here just to check if the attack worked
len(potential_private_keys)

Decrypted, attack worked!


In [11]:
wiener_atk_d = check_private_key(potential_private_keys, e, n)

In [12]:
print(decrypt(encrypted_message, n, wiener_atk_d))

Cryptography is the practice and study of techniques for secure communication in the presence of third parties called adversaries.


#### More precise bound

In [14]:
e, n, d, _, _, _ = gen_vulnerable_keys_v2(nbits=1024, s=15)
message = "Cryptography is the practice and study of techniques for secure communication in the presence of third parties called adversaries."
encrypted_message = encrypt(message, n, e)
print(encrypted_message)

...
Decrypted, attack worked!


385

In [9]:
potential_private_keys = wiener_atk(e, n, d) # d is here just to check if the attack worked
len(potential_private_keys)

Decrypted, attack worked!


392

In [4]:
wiener_atk_d = check_private_key(potential_private_keys, e, n)

In [5]:
print(decrypt(encrypted_message, n, wiener_atk_d))

Cryptography is the practice and study of techniques for secure communication in the presence of third parties called adversaries.
